In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import rdata

In [2]:
data_path = "data.Rdata" ; otu_path = "otu.Rdata"
converted_data = rdata.read_rds(data_path) ; converted_otu = rdata.read_rds(otu_path)
data_df = pd.DataFrame(converted_data['meta.full']) ; otu_df = pd.DataFrame(converted_data['meta.full'])
print(f"Data_df ---> {data_df.shape}")
print(f"Otu_df ---> {otu_df.shape}")

C:\Users\Marc2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\rdata\parser\_parser.py:1239: UserWarning: Wrong extension .rds for file in RDATA format
  warnings.warn(  # noqa: B028


Data_df ---> (284, 61)
Otu_df ---> (284, 61)


C:\Users\Marc2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\rdata\parser\_parser.py:1239: UserWarning: Wrong extension .rds for file in RDATA format
  warnings.warn(  # noqa: B028


## Preprocessing


### DATA_DF
- Suppression des colonnes ayant au plus deux valeurs uniques.
- Suppression également des colonnes ou on a comme valeur `Missing: Not provided` ou `Missing: Restricted access`.
- Sur la colonne `sample_name`, on remarque que le début de l'identifiant est toujours identique, on peut juste prendre la fin.
- Même chose pour la colonne `anonymized_name`, on peut juste prendre la fin de l'identifiant.
- Pour la colonne `description`, on peut juste prendre le milieu de l'identifiant.
- Pour la colonne `orig_name`, on peut juste prendre la fin de l'identifiant.
- Pour la colonne `host_subject_id`, on peut juste prendre la fin de l'identifiant.
- Drop la colonne `gastrointest_disord` car elle est identique a diagnosis.
- Mise en miniscule des str.
- Suppression des lignes ayant au moins un nan.
- Garder les `no` (pour faire une  classification `malades` vs `non malades`).
- Ou (Non malade vs les différents type de maladies).
- Drop les lignes ou on a au moins un `no` dans `diagnosis` ou dans `diseasesubtype`.
- Faire un `OneHotEncoder` pour les variables catégorielles.

### OTU_DF
- Diviser chacune des variables de la ligne par la somme.
- Remplacer par des `0` les valeurs manquantes.



- Dans certains modèles (utiliser present/pas_present)

In [ ]:
def purge_df(df : pd.DataFrame) -> pd.DataFrame : 
    """
    Fonction de faire le préprocessing d'un dataframe.
    
    Paramètres : 
        df (pd.DataFrame) : DataFrame avant la purge.
    
    Retour : 
        result_df (pd.DataFrame) : DataFrame après la purge.
    """
    
    
    # Suppression des colonnes ayant au plus deux valeurs uniques
    counts_by_col = []
    for column in df_clean.columns : 
        unique_values = df_clean[column].unique()
        #print(f"La colonne {column} a {len(unique_values)} valeurs ")
        if len(unique_values) > 2: 
            counts_by_col.append({column : len(unique_values)})

    remaining_columns = [list(el.keys())[0] for el in counts_by_col]
    df_clean = df.loc[:,remaining_columns]

    # Suppression également des colonnes ou on a comme valeur "Missing: Not provided" ou "Missing: Restricted access"
    phrases = ['Missing: Not provided', 'Missing: Restricted access']
    text_cols = df_clean.select_dtypes(include=['object', 'string', 'category']).columns
    cols_to_drop = df_clean[text_cols].apply(lambda s: s.isin(phrases)).any()
    df_clean = df_clean.drop(columns=cols_to_drop[cols_to_drop].index)

    # Sur la colonne sample_name, on remarque que le début de l'identifiant est toujours identique, on peut juste prendre la fin
    df_clean["sample_name"] = df_clean["sample_name"].apply(lambda s : str.split(s,'.')[-1])

    # Même chose pour la colonne anonymized_name, on peut juste prendre la fin de l'identifiant
    df_clean["anonymized_name"] = df_clean["anonymized_name"].apply(lambda s : str.split(s,'.')[-1])

    # Pour la colonne description, on peut juste prendre le milieu de l'identifiant
    df_clean['description'] = df_clean["description"].apply(lambda s : str.split((str.split(s,'-')[-1])," ")[0])

    # Pour la colonne orig_name, on peut juste prendre la fin de l'identifiant
    df_clean['orig_name']  = df_clean["orig_name"].apply(lambda s : str.split(s,'.')[-1])

    # Pour la colonne host_subject_id, on peut juste prendre la fin de l'identifiant
    df_clean['host_subject_id']  = df_clean["host_subject_id"].apply(lambda s : str.split(s,'-')[-1])

    # Drop la colonne gastrointest_disord car elle est identique a diagnosis
    df_clean = df_clean.drop(columns="gastrointest_disord",axis=1)
    
    # Changement du label "no" en "unidentified" et mise en miniscule des str
    df_clean["diagnosis"] = df_clean["diagnosis"].apply(lambda s : s.lower())
    df_clean["diseasesubtype"] = df_clean["diseasesubtype"].apply(lambda s : s.lower())
    
    # Suppression des lignes ayant au moins un nan     
    df_clean = df_clean[~df_clean.isna().any(axis=1)] 
    
    # Drop les lignes ou on a au moins un "no" dans 'diagnosis' ou dans 'diseasesubtype'
    cols = ['diagnosis', 'diseasesubtype']

    rows_to_drop = df_clean[cols].apply(lambda s: s.eq('no')).any(axis=1)

    df_clean = df_clean.loc[~rows_to_drop].reset_index(drop=True)
    
    # Garder age et diagnosis
    df_clean = df_clean.loc[:,["age","diagnosis"]]
    
    return df_clean
# # Suppression des valeurs abérrantes
# q1 = data_df_clean.quantile(0.25)
# q3 = data_df_clean.quantile(0.75)
# iqr = q3 - q1
# lower = q1 - 1.5 * iqr
# upper = q3 + 1.5 * iqr
# out = data_df_clean.lt(lower, axis=1) | data_df_clean.gt(upper, axis=1)
# out


In [4]:
data_df_clean =purge_df(data_df)
data_df_clean
# data_df_clean["diagnosis"].value_counts().plot.bar()
# plt.plot()
# plt.show()

,sample_name,age,anonymized_name,description,diagnosis,diseasesubtype,host_subject_id,orig_name
0,0448,11.500000,0448,0181,cd,ccd,0181,0448
1,0766,8.500000,0766,0563,cd,icd,0563,0766
2,0877,16.833333,0877,0110,ic,ic,0110,0877
3,0890,13.750000,0890,0374,uc,uc,0374,0890
4,0937,11.166667,0937,0309,cd,icd,0309,0937
...,...,...,...,...,...,...,...,...
177,1268,15.500000,1268,0322,uc,uc,0322,1268
178,0749,15.750000,0749,0232,cd,icd,0232,0749
179,0987,13.583333,0987,0812,uc,uc,0812,0987
180,1103,8.833333,1103,0624,cd,icd,0624,1103
